In [2]:
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import geopandas as gpd
from libpysal.weights import KNN
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm 

In [ ]:
# Load dataset (Manually set the file path for each category)
file_path = f"new_dataset\Reptilia_top_species.csv"  # Change this for different categories
df = pd.read_csv(file_path)

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\sanff\AppData\Local\Temp\ipykernel_8452\4204212959.py:2: SyntaxWarning: invalid escape sequence '\R'
  file_path = f"new_dataset\Reptilia_top_species.csv"  # Change this for different categories


In [4]:
print(df["scientific_name"].unique())  # Check the unique scientific names in the dataset

['calotes versicolor' 'fowlea piscator' 'hemidactylus']


In [5]:
# Group by lat/lon and count how many unique species were observed at each location
unique_species_per_location = df.groupby(["latitude", "longitude"])["scientific_name"].nunique().reset_index()
print(unique_species_per_location)
print("----------------------------------------------------------------------------")
# Now filter locations where more than 1 species was observed
multi_species_locations = unique_species_per_location[unique_species_per_location["scientific_name"] > 1]
print(multi_species_locations)
print(unique_species_per_location["scientific_name"].unique())  # Check the unique counts of species per location
print("----------------------------------------------------------------------------")

print(f"Number of unique lat/lon pairs with multiple species: {len(multi_species_locations)}")
multi_species_locations.head()


       latitude  longitude  scientific_name
0      8.083090  77.546730                1
1      8.088570  77.506172                1
2      8.095061  77.446014                1
3      8.121021  77.483579                1
4      8.125228  77.566278                1
...         ...        ...              ...
9715  32.719370  74.868063                1
9716  32.726602  74.857026                1
9717  32.747506  74.855181                1
9718  32.774579  74.839565                1
9719  33.065206  75.308792                1

[9720 rows x 3 columns]
----------------------------------------------------------------------------
       latitude  longitude  scientific_name
9      8.180995  77.403749                2
11     8.183284  77.411900                3
12     8.183286  77.411900                2
69     8.433417  77.401340                2
83     8.453915  76.973293                2
...         ...        ...              ...
9300  28.942339  79.765045                2
9352  28.947522  7

,latitude,longitude,scientific_name
9,8.180995,77.403749,2
11,8.183284,77.411900,3
12,8.183286,77.411900,2
69,8.433417,77.401340,2
83,8.453915,76.973293,2


In [9]:
# Count how many times the same species was seen at same lat/lon (could be on different days)
repeat_species = df.groupby(["latitude", "longitude", "scientific_name"]).size().reset_index(name="count")

# Filter where count > 1 to find duplicates
repeat_species_obs = repeat_species[repeat_species["count"] > 1]
print(repeat_species["count"].unique())  # Check the unique counts of species per location

print(f"Same species observed multiple times at same lat/lon: {len(repeat_species_obs)}")
repeat_species_obs.head()


[ 1 11  3  4  2  5  8 37  7  9 12  6 10 18 17 15 13 24]
Same species observed multiple times at same lat/lon: 633


,latitude,longitude,scientific_name,count
9,8.180995,77.403749,calotes versicolor,11
14,8.183284,77.411900,hemidactylus,3
19,8.186770,77.424704,hemidactylus,4
92,8.461115,77.481737,hemidactylus,2
95,8.464616,76.927576,calotes versicolor,5


In [7]:
# Count how many unique dates each species was observed at each location
species_date_check = df.groupby(["latitude", "longitude", "scientific_name"])["observed_date"].nunique().reset_index(name="unique_dates")

# Filter to see species observed multiple times (on different dates)
species_multiple_dates = species_date_check[species_date_check["unique_dates"] > 1]

print(f"Species observed multiple times at same location on different dates: {len(species_multiple_dates)}")
species_multiple_dates.head()


Species observed multiple times at same location on different dates: 447


,latitude,longitude,scientific_name,unique_dates
9,8.180995,77.403749,calotes versicolor,8
14,8.183284,77.411900,hemidactylus,2
19,8.186770,77.424704,hemidactylus,3
92,8.461115,77.481737,hemidactylus,2
95,8.464616,76.927576,calotes versicolor,4


In [8]:
# Find repeated species at same location on same date
duplicates_check = df.groupby(["latitude", "longitude", "scientific_name", "observed_date"]).size().reset_index(name="count")
potential_duplicates = duplicates_check[duplicates_check["count"] > 1]

print(f"Potential duplicate entries (same species, same location, same date): {len(potential_duplicates)}")
potential_duplicates.head()


Potential duplicate entries (same species, same location, same date): 323


,latitude,longitude,scientific_name,observed_date,count
11,8.180995,77.403749,calotes versicolor,2015-07-01,2
13,8.180995,77.403749,calotes versicolor,2015-07-11,2
16,8.180995,77.403749,calotes versicolor,2019-02-08,2
21,8.183284,77.411900,hemidactylus,2015-07-13,2
28,8.186770,77.424704,hemidactylus,2017-11-30,2
